# Natural Transformations

In Category Theory, a category $\mathcal C$ has objects and morphisms, which act like functions
between objects. In similar fashion, we can define "functions" between categories,
which we call functors. These functors preserve the composition between the categories, for example,
for $F:\mathcal C \to \mathcal D$, for morphisms $f:a \to b \in \mathcal C(A,B)$
and $g: B\to C \in \mathcal C(B,C)$, we have that
$Fg \circ Ff = F(f \circ g)$.
Also, functors must take identity morphisms to identity morphisms, i.e. $F id_A = id_{FA}$.

Hence, if functors are like morphisms between categories, natural transformations act as
morphisms between functors.
Formally:

**Definition: Natural Transformations (Category Theory)**. Let $\mathcal C$ and $\mathcal D$
be categories, and let $F,G$ be functors from $\mathcal C$ to $\mathcal D$. A natural
transformation $\alpha : \mathcal C \Rightarrow \mathcal D$ is such that:
* For all $A \in \mathcal C$, $\alpha_A: FA \to GA$ is a morphism in $\mathcal D$;
* For all $f \in \mathcal C(A,B)$,
$$
\alpha_B \circ Ff = Gf \circ \alpha_A
$$

## 1. Natural Transformations in Programming

<!-- As we've said before, in programming, there is a main category $\mathcal C$
where types are objects and functions are morphisms. Also, we can define endofunctors (i.e. $F:\mathcal C \to \mathcal C$) -->

The formal definition looks cryptic, but it's not so complex as it looks at first.
Note that, in programming, a natural transformation is a parametric function. Meaning,
for every type `T`, we have a function `α{T}`.

Below we present a simple example of a parametric function:

In [1]:
struct α{T} end

α{Real}(x::Real) = x^2
α{String}(x::String) = x*x
α{T}(x::T) where T =  x
α{Real}(10), α{String}("A"), α{Any}((1,2))

(100, "AA", (1, 2))

Note that the parametric function we defined was somewhat redundant in terms of Julia. Why? Because
in Julia we have multiple dispatch. Hence, we could've totally ommited the parameter type, or,
we could've provided it as an argument, and the same would've been achieved. 
See below:

In [2]:
β(x::Real) = x^2
β(x::String) = x * x
β(x::Any) = x

β(10), β("A"), β((1,2))

(100, "AA", (1, 2))

Now, the natural transformation is not simply a parametric function. For it be actually
be a natural transformation between two functors, it's necessary to satisfy
the condtition that for `f(x::A)::B` and `a::F(A)`, then
`α{B}(fmap(f,a)) =  fmap(f,α{A}(a))`.

**Word of caution!** Note that in Julia, we can define a parametric function to a restricted number of types. By doing so,
our parametric type will *not* be a natural transformation, since a natural transformation requires to define a function
for each object in our category $\textbf{Types}$. In Haskell, polymorphic functions are always defined to every possible type, so this
is not something to worry; Yet, this is not the case for Julia.

## 2. First Example of Natural Transformation

Let's begin with our first example. Consider the functors `Maybe` and the `List` functor we've implemented in the section of Functors.


In [1]:
abstract type Maybe{T} end
struct Just{T} <: Maybe{T}
    _1::T
end
struct Null{T} <: Maybe{T} end

Maybe(T::Type) = Maybe{T}
fmap(f::Function, x::Maybe{T}) where T = x isa Null ? Null{T}() : Just(f(x._1))


abstract type List{T} end
struct Nil{T} <: List{T} end
struct Cons{T} <: List{T}
  val::T
  next::List{T} # n.b. abstract type!
end
Nil(T) = Nil{T}()
Nil() = Nil(Any)

Cons(x::T) where T = Cons(x, Nil(T))
# l = Cons(1,Cons(2));

fmap(f::Function, x::Nil) = x
fmap(f::Function, x::Cons{T}) where T = Cons(f(x.val),fmap(f, x.next))

fmap (generic function with 3 methods)

Let's define a natural transformation between these two functors. 

In [4]:
safehead(x::Nil) =  Null{List}()
safehead(x::Cons) = Just(x.val)

l = Cons(1,Cons(2));
safehead(l), safehead(Nil())

(Just{Int64}(1), Null{List}())

Let's check if the conditions are satisfied.  The condition
just means that we can either apply the `safehead` function after or before apply the
`fmap(f,)`.

In [12]:
f(x) = x^2

@show safehead(fmap(f, l)) == fmap(f,safehead(l))
safehead(fmap(f, l))

safehead(fmap(f, l)) == fmap(f, safehead(l)) = true


Just{Int64}(1)